In [29]:
import requests
import pandas as pd
from datetime import datetime

# Etherscan API key
api_key = '1RY32CCK7M9Z9AQPFZIMXB6HZE8PW2XMDN'

# Function to get Ethereum transactions
def get_ethereum_transactions(address, start_block, end_block, total_transactions=500):
    all_transactions = []
    page = 1
    offset = 100  # Increase the number of records per request to 100

    while len(all_transactions) < total_transactions:
        url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock={start_block}&endblock={end_block}&page={page}&offset={offset}&apikey={api_key}"
        
        response = requests.get(url)
        data = response.json()
        
        if data['status'] == '1' and data['message'] == 'OK':
            transactions = data['result']
            all_transactions.extend(transactions)
            page += 1
        else:
            print(f"Error fetching data: {data['message']}")
            break
    
    return pd.DataFrame(all_transactions[:total_transactions])

# Function to get ERC20 transactions
def get_erc20_transactions(address, start_block, end_block, total_transactions=500):
    all_transactions = []
    page = 1
    offset = 100  # Increase the number of records per request to 100

    while len(all_transactions) < total_transactions:
        url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&startblock={start_block}&endblock={end_block}&page={page}&offset={offset}&apikey={api_key}"
        
        response = requests.get(url)
        data = response.json()
        
        if data['status'] == '1' and data['message'] == 'OK':
            transactions = data['result']
            all_transactions.extend(transactions)
            page += 1
        else:
            print(f"Error fetching data: {data['message']}")
            break
    
    return pd.DataFrame(all_transactions[:total_transactions])

# Function to get account balance
def get_account_balance(address):
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == '1' and data['message'] == 'OK':
        balance = float(data['result']) / 10**18  # Convert balance to Ether
    else:
        print(f"Error fetching balance: {data['message']}")
        balance = 0.0
    
    return balance

# Function to calculate unique from addresses for each transaction
def calculate_unique_from_addresses(transactions_df):
    unique_from_counts = transactions_df['from'].nunique()
    transactions_df['Unique Received From Addresses'] = unique_from_counts - 1
    return transactions_df

# Function to calculate time difference between first and last transaction in minutes
def calculate_time_difference(transactions_df):
    if 'timeStamp' in transactions_df.columns:
        transactions_df['timeStamp'] = pd.to_datetime(transactions_df['timeStamp'], unit='s')
        first_transaction_time = transactions_df['timeStamp'].min()
        last_transaction_time = transactions_df['timeStamp'].max()
        time_diff_minutes = (last_transaction_time - first_transaction_time).total_seconds() / 60
        transactions_df['Time Diff between first and last (Mins)'] = time_diff_minutes
    return transactions_df

# Function to calculate additional features
# def calculate_additional_features(transactions_df, erc20_transactions_df, address):
#     # Initialize default values
#     erc20_max_val_rec = 0
#     erc20_min_val_rec = 0
#     erc20_avg_val_rec = 0
#     erc20_total_ether_received = 0
#     erc20_uniq_rec_token_name = 0
#     erc20_uniq_rec_contract_addr = 0
#     total_erc20_tnxs = 0
#     erc20_max_val_sent = 0
#     erc20_avg_val_sent = 0
#     erc20_total_ether_sent = 0
#     erc20_uniq_sent_token_name = 0
#     erc20_uniq_sent_addr = 0
#     avg_min_between_received_tnx = 0
#     avg_value_received = 0
#     total_ether_received = 0
#     min_value_received = 0
#     max_value_received = 0
#     received_tnx = 0
#     min_val_sent = 0
#     max_val_sent = 0
#     total_ether_sent = 0
#     unique_sent_to_addresses = 0
#     number_of_created_contracts = 0

#     # ERC20_Max_Val_Rec
#     if not erc20_transactions_df.empty:
#         erc20_received = erc20_transactions_df[erc20_transactions_df['to'].str.lower() == address.lower()]
#         erc20_sent = erc20_transactions_df[erc20_transactions_df['from'].str.lower() == address.lower()]
        
#         if not erc20_received.empty:
#             erc20_received['value'] = erc20_received['value'].astype(float) / 10**18  # Convert to Ether
#             erc20_max_val_rec = erc20_received['value'].max()
#             erc20_min_val_rec = erc20_received['value'].min()
#             erc20_avg_val_rec = erc20_received['value'].mean()
#             erc20_total_ether_received = erc20_received['value'].sum()
#             erc20_uniq_rec_token_name = erc20_received['tokenName'].nunique()
#             erc20_uniq_rec_contract_addr = erc20_received['contractAddress'].nunique()
#             total_erc20_tnxs = len(erc20_received)
        
#         if not erc20_sent.empty:
#             erc20_sent['value'] = erc20_sent['value'].astype(float) / 10**18  # Convert to Ether
#             erc20_max_val_sent = erc20_sent['value'].max()
#             erc20_avg_val_sent = erc20_sent['value'].mean()
#             erc20_total_ether_sent = erc20_sent['value'].sum()
#             erc20_uniq_sent_token_name = erc20_sent['tokenName'].nunique()
#             erc20_uniq_sent_addr = erc20_sent['to'].nunique()
    
#     # Total_Transactions(Including_Tnx_to_Create_Contract)
#     total_transactions = len(transactions_df)
    
#     # Avg_min_between_received_tnx
#     if not transactions_df.empty:
#         received_transactions = transactions_df[transactions_df['to'].str.lower() == address.lower()]
#         if not received_transactions.empty:
#             received_transactions['value'] = received_transactions['value'].astype(float) / 10**18  # Convert to Ether
#             avg_min_between_received_tnx = received_transactions['timeStamp'].diff().mean().total_seconds() / 60
#             avg_value_received = received_transactions['value'].mean()
#             total_ether_received = received_transactions['value'].sum()
#             min_value_received = received_transactions['value'].min()
#             max_value_received = received_transactions['value'].max()
#             received_tnx = len(received_transactions)

#         # Sent transactions
#         sent_transactions = transactions_df[transactions_df['from'].str.lower() == address.lower()]
#         if not sent_transactions.empty:
#             sent_transactions['value'] = sent_transactions['value'].astype(float) / 10**18  # Convert to Ether
#             min_val_sent = sent_transactions['value'].min()
#             max_val_sent = sent_transactions['value'].max()
#             total_ether_sent = sent_transactions['value'].sum()
#             unique_sent_to_addresses = sent_transactions['to'].nunique()
    
#         # Number_of_Created_Contracts
#         number_of_created_contracts = len(transactions_df[transactions_df['isError'] == '0'])

#     # Total_Ether_Balance
#     total_ether_balance = get_account_balance(address)

#     transactions_df[' ERC20 max val rec'] = erc20_max_val_rec
#     transactions_df[' Total ERC20 tnxs'] = total_erc20_tnxs
#     transactions_df['total transactions (including tnx to create contract'] = total_transactions
#     transactions_df[' ERC20 total Ether received'] = erc20_total_ether_received
#     transactions_df['total ether balance'] = total_ether_balance
#     transactions_df['avg val received'] = avg_value_received
#     transactions_df['total ether received'] = total_ether_received
#     transactions_df['Avg min between received tnx'] = avg_min_between_received_tnx
#     transactions_df[' ERC20 total ether sent'] = erc20_total_ether_sent
#     transactions_df[' ERC20 avg val rec'] = erc20_avg_val_rec
#     transactions_df['Received Tnx'] = received_tnx
#     transactions_df['min val sent'] = min_val_sent
#     transactions_df[' ERC20 min val rec'] = erc20_min_val_rec
#     transactions_df['min value received'] = min_value_received
#     transactions_df[' ERC20 uniq rec contract addr'] = erc20_uniq_rec_contract_addr
#     transactions_df['max value received '] = max_value_received
#     transactions_df['total Ether sent'] = total_ether_sent
#     transactions_df[' ERC20 uniq sent token name'] = erc20_uniq_sent_token_name
#     transactions_df[' ERC20 uniq sent addr'] = erc20_uniq_sent_addr
#     transactions_df['max val sent'] = max_val_sent
#     transactions_df[' ERC20 avg val sent'] = erc20_avg_val_sent
#     transactions_df['Unique Sent To Addresses'] = unique_sent_to_addresses
#     transactions_df[' ERC20 max val sent'] = erc20_max_val_sent
#     transactions_df[' ERC20 uniq rec token name'] = erc20_uniq_rec_token_name
#     transactions_df['Number of Created Contracts'] = number_of_created_contracts
#     transactions_df['Avg min between sent tnx']=0
#     transactions_df['Sent tnx'] = 0
#     transactions_df['avg val sent'] = 0
#     transactions_df['min value sent to contract'] = 0
#     transactions_df['max val sent to contract'] = 0
#     transactions_df['avg value sent to contract'] = 0
#     transactions_df['total ether sent contracts'] = 0
#     transactions_df[' ERC20 total Ether sent contract'] = 0
#     transactions_df[' ERC20 uniq rec addr'] = 0
#     return transactions_df

def calculate_additional_features(transactions_df, erc20_transactions_df, address):
    # 初始化所有需要的特征
    features = {
        " ERC20 max val rec": 0,
        " Total ERC20 tnxs": 0,
        "total transactions (including tnx to create contract": len(transactions_df),
        " ERC20 total Ether received": 0,
        "total ether balance": get_account_balance(address),
        "avg val received": 0,
        "total ether received": 0,
        "Avg min between received tnx": 0,
        " ERC20 total ether sent": 0,
        " ERC20 avg val rec": 0,
        "Received Tnx": 0,
        "min val sent": 0,  # 
        " ERC20 min val rec": 0,
        "min value received": 0,
        " ERC20 uniq rec contract addr": 0,
        "max value received ": 0,
        "total Ether sent": 0,
        " ERC20 uniq sent token name": 0,
        " ERC20 uniq sent addr": 0,
        "max val sent": 0,
        " ERC20 avg val sent": 0,
        "Unique Sent To Addresses": 0,
        " ERC20 max val sent": 0,
        " ERC20 uniq rec token name": 0,
        "Number of Created Contracts": 0,
        "Avg min between sent tnx": 0,
        "Sent tnx": 0,
        "avg val sent": 0,
        "min value sent to contract": 0,
        "max val sent to contract": 0,
        "avg value sent to contract": 0,
        "total ether sent contracts": 0,
        " ERC20 total Ether sent contract": 0,
        " ERC20 uniq rec addr": 0
    }

    # 处理收到的ERC20交易
    if 'to' in erc20_transactions_df.columns:
        erc20_received = erc20_transactions_df[erc20_transactions_df['to'].str.lower() == address.lower()]
        if not erc20_received.empty:
            erc20_received['value'] = erc20_received['value'].astype(float) / 10**18  # Convert to Ether
            features[" ERC20 max val rec"] = erc20_received['value'].max()
            features[" ERC20 min val rec"] = erc20_received['value'].min()
            features[" ERC20 avg val rec"] = erc20_received['value'].mean()
            features[" ERC20 total Ether received"] = erc20_received['value'].sum()
            features[" ERC20 uniq rec token name"] = erc20_received['tokenName'].nunique()
            features[" ERC20 uniq rec contract addr"] = erc20_received['contractAddress'].nunique()
            features[" Total ERC20 tnxs"] = len(erc20_received)
            features[" ERC20 uniq rec addr"] = erc20_received['from'].nunique()

    # 处理发送的ERC20交易
    if 'from' in erc20_transactions_df.columns:
        erc20_sent = erc20_transactions_df[erc20_transactions_df['from'].str.lower() == address.lower()]
        if not erc20_sent.empty:
            erc20_sent['value'] = erc20_sent['value'].astype(float) / 10**18  # Convert to Ether
            features[" ERC20 max val sent"] = erc20_sent['value'].max()
            features[" ERC20 avg val sent"] = erc20_sent['value'].mean()
            features[" ERC20 total ether sent"] = erc20_sent['value'].sum()
            features[" ERC20 uniq sent token name"] = erc20_sent['tokenName'].nunique()
            features[" ERC20 uniq sent addr"] = erc20_sent['to'].nunique()
            features[" ERC20 total Ether sent contract"] = erc20_sent['value'].sum()  # 假设所有发送到ERC20合约

    # 处理普通交易
    if 'from' in transactions_df.columns:
        sent_transactions = transactions_df[transactions_df['from'].str.lower() == address.lower()]
        if not sent_transactions.empty:
            sent_transactions['value'] = sent_transactions['value'].astype(float) / 10**18  # Convert to Ether
            features["Avg min between sent tnx"] = sent_transactions['timeStamp'].diff().mean().total_seconds() / 60
            features["Sent tnx"] = len(sent_transactions)
            features["avg val sent"] = sent_transactions['value'].mean()
            features["min val sent"] = sent_transactions['value'].min()
            features["max val sent"] = sent_transactions['value'].max()
            features["total Ether sent"] = sent_transactions['value'].sum()
            features["Unique Sent To Addresses"] = sent_transactions['to'].nunique()

            # 假设有发送到合约的交易，需要实际数据结构确认
            sent_to_contracts = sent_transactions[sent_transactions['to'].apply(lambda x: x.startswith('0x'))]  # 假设合约地址以'0x'开始
            if not sent_to_contracts.empty:
                features["min value sent to contract"] = sent_to_contracts['value'].min()
                features["max val sent to contract"] = sent_to_contracts['value'].max()
                features["avg value sent to contract"] = sent_to_contracts['value'].mean()
                features["total ether sent contracts"] = sent_to_contracts['value'].sum()

    # Number_of_Created_Contracts
    features["Number of Created Contracts"] = len(transactions_df[transactions_df['isError'] == '0'])

    # Update the transactions DataFrame with calculated features
    for key, value in features.items():
        transactions_df[key] = value

    return transactions_df

# Example: Get transactions for a specific address
address = '0x03d42acad7cdac001c80eb1443c24f18893eb2d0'
start_block = 0
end_block = 99999999
transactions = get_ethereum_transactions(address, start_block, end_block, total_transactions=2500)
erc20_transactions = get_erc20_transactions(address, start_block, end_block, total_transactions=2500)

# Calculate unique from addresses, time difference, and additional features
if not transactions.empty:
    transactions = calculate_unique_from_addresses(transactions)
    transactions = calculate_time_difference(transactions)
    transactions = calculate_additional_features(transactions, erc20_transactions, address)
    csv_file = 'ethereum_transactions_with_additional_features.csv'
    transactions.to_csv(csv_file, index=False)
    print(f"Transaction data with additional features has been saved to '{csv_file}'.")
else:
    print("No transaction data retrieved.")

# Load the data back into a DataFrame
loaded_transactions = pd.read_csv(csv_file)
print("Loaded Transactions DataFrame with additional features:")
print(loaded_transactions)


Error fetching data: No transactions found
Error fetching data: No transactions found


C:\Users\MartinQ\AppData\Local\Temp\ipykernel_32944\1418178306.py:75: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  transactions_df['timeStamp'] = pd.to_datetime(transactions_df['timeStamp'], unit='s')


Transaction data with additional features has been saved to 'ethereum_transactions_with_additional_features.csv'.
Loaded Transactions DataFrame with additional features:
     blockNumber            timeStamp  \
0        3792639  2017-05-30 12:30:56   
1        3792741  2017-05-30 12:52:16   
2        3795984  2017-05-31 02:52:48   
3        3796011  2017-05-31 02:59:12   
4        3829146  2017-06-06 11:09:52   
..           ...                  ...   
260     11508355  2020-12-23 07:08:48   
261     11508510  2020-12-23 07:42:56   
262     13522501  2021-10-31 03:41:52   
263     13567983  2021-11-07 07:13:04   
264     13568193  2021-11-07 08:04:16   

                                                  hash   nonce  \
0    0xf24d9ac20382f3e267bd16253e81ce48c1fc99fea1aa...  388100   
1    0x6c161003289ff237121c6ac69e60f41f6e0d30ae0b63...   57032   
2    0x85bdae4f9a4cad894e39fa3f6bdbb5f7249cb2933bd7...       0   
3    0xea8d8f94cb7a53c8a8b464fbb8cc841e5479854c8580...       7   
4    0x

C:\Users\MartinQ\AppData\Local\Temp\ipykernel_32944\1418178306.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erc20_received['value'] = erc20_received['value'].astype(float) / 10**18  # Convert to Ether
C:\Users\MartinQ\AppData\Local\Temp\ipykernel_32944\1418178306.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erc20_sent['value'] = erc20_sent['value'].astype(float) / 10**18  # Convert to Ether
C:\Users\MartinQ\AppData\Local\Temp\ipykernel_32944\1418178306.py:267: SettingWithCopyWarning: 
A va

In [30]:
loaded_transactions

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,Number of Created Contracts,Avg min between sent tnx,Sent tnx,avg val sent,min value sent to contract,max val sent to contract,avg value sent to contract,total ether sent contracts,ERC20 total Ether sent contract,ERC20 uniq rec addr
0,3792639,2017-05-30 12:30:56,0xf24d9ac20382f3e267bd16253e81ce48c1fc99fea1aa...,388100,0x695975e2ab2a4c2ac2d74cd7862a3eb9b45270a42be2...,32,0x32be343b94f860124dc4fee278fdcbd38c102d88,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,30008995470000000000,333333,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
1,3792741,2017-05-30 12:52:16,0x6c161003289ff237121c6ac69e60f41f6e0d30ae0b63...,57032,0x62b84bc5156524321d306453d66137368e9cf337b9df...,45,0xd94c9ff168dc6aebf9b6cc86deff54f3fb0afc33,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,64000000000000000000,90000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
2,3795984,2017-05-31 02:52:48,0x85bdae4f9a4cad894e39fa3f6bdbb5f7249cb2933bd7...,0,0x5b09a17867e561187035f1f401d6ce9e8ec7ad2a2f38...,0,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x6718a270a7f28a59db445f12333bd5d0dc327b4f,1000000000000000000,250000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
3,3796011,2017-05-31 02:59:12,0xea8d8f94cb7a53c8a8b464fbb8cc841e5479854c8580...,7,0xfaac92bbade48975c932e26282356274451d1219cf31...,15,0x6718a270a7f28a59db445f12333bd5d0dc327b4f,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,1000000000000000000,23101,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
4,3829146,2017-06-06 11:09:52,0xab52963f3bfe57b7862fd844499a6699542d3aeb5bfc...,1,0x0ab064267a1715de2127e1d8530f4be732d6274072f5...,4,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x61b92ec8b6f56d39af3e891e2d33aa36a7e77049,60000000000000000000,60000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,11508355,2020-12-23 07:08:48,0x7f2e619a494ca3e6ee3e3c8901d1e4878a47eefcb599...,152,0x6dc38b049f1fd7e824a2396329d0ca98052b744e53d8...,155,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x0000000006a0403952389b70d8ee4e45479023db,0,119928,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
261,11508510,2020-12-23 07:42:56,0x385f411dbcdb5416f4bef4d5a3aff6a2b02622b2f3d8...,153,0x3c8138d9a59c92ba95fe906bddf6af1f9112234e9628...,182,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x0000000000095413afc295d19edeb1ad7b71c952,0,60000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
262,13522501,2021-10-31 03:41:52,0x4d9b4e1b5f89ac45dc8e8859e95b4fabc98dcc576f0f...,154,0x9d7d589ae7e4941a34cdaa61ef3cd25e35ce36d3c8d4...,286,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x76960dccd5a1fe799f7c29be9f19ceb4627aeb2f,0,80000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83
263,13567983,2021-11-07 07:13:04,0x3e5f8b1ee6cb18cb33aa9407d870f60dd45211aacc49...,155,0xd8f4eac818aee399b90945b43dbb4864fd08a3cd7f83...,242,0x03d42acad7cdac001c80eb1443c24f18893eb2d0,0x58b6a8a3302369daec383334672404ee733ab239,0,80000,...,264,14965.073504,157,29.958406,0.0,1000.0,29.958406,4703.469763,1.401136e+07,83


## 更新数据

In [31]:
# 加载用户上传的文件
uploaded_file_path = './modified_mean_values_dataset.csv'
df_modified = pd.read_csv(uploaded_file_path)

# 提取最后一行数据

last_row = loaded_transactions.iloc[-1]
last_row


blockNumber                                                                                      13568193
timeStamp                                                                             2021-11-07 08:04:16
hash                                                    0x8f43ee6047f8ff4aeeb2a9890d9e35f4ae425620573b...
nonce                                                                                                 156
blockHash                                               0x004165c96acaf9a106dd94172fb937907f88e39da680...
transactionIndex                                                                                      228
from                                                           0x03d42acad7cdac001c80eb1443c24f18893eb2d0
to                                                             0x430c7bc95b99aeaaeb5a63e84a2ea0586ee252b7
value                                                                                   66600000000000000
gas                                           

In [32]:
df_modified

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,4730,0xNewAddressForTestWithMeanValues,0,5086.878721,8004.851184,218333.257652,115.931714,163.700945,3.729702,...,11741.257297,1.303594e+07,6.318389e+06,0.0,0.0,0.0,1.384931,4.826676,NaN,NaN


In [33]:
# 假设要替换的目标文件是之前生成的 `df_mean_complete`
# 如果df_mean_complete尚未生成，您可以从现有的DataFrame重新加载
# df_mean_complete_path = '/mnt/data/mean_values_dataset.csv'
# df_mean_complete = pd.read_csv(df_mean_complete_path)

# 替换 `df_mean_complete` 中对应列的值
count = 0
for column in df_modified.columns:
    if column in last_row.index:
        print(column)
        count+=1 
        df_modified.at[0, column] = last_row[column]
        
df_modified = df_modified.drop(columns=['Unnamed: 0'])
# 保存修改后的DataFrame到新的CSV文件
updated_df_path = './updated_mean_values_dataset.csv'
df_modified.to_csv(updated_df_path, index=True)

count

Avg min between sent tnx
Avg min between received tnx
Time Diff between first and last (Mins)
Sent tnx
Received Tnx
Number of Created Contracts
Unique Received From Addresses
Unique Sent To Addresses
min value received
max value received 
avg val received
min val sent
max val sent
avg val sent
min value sent to contract
max val sent to contract
avg value sent to contract
total transactions (including tnx to create contract
total Ether sent
total ether received
total ether sent contracts
total ether balance
 Total ERC20 tnxs
 ERC20 total Ether received
 ERC20 total ether sent
 ERC20 total Ether sent contract
 ERC20 uniq sent addr
 ERC20 uniq rec addr
 ERC20 uniq rec contract addr
 ERC20 min val rec
 ERC20 max val rec
 ERC20 avg val rec
 ERC20 max val sent
 ERC20 avg val sent
 ERC20 uniq sent token name
 ERC20 uniq rec token name


36

In [34]:
df_modified.shape

(1, 50)

In [35]:
df_modified

,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,4730,0xNewAddressForTestWithMeanValues,0,14965.073504,0.0,2.335413e+06,157.0,0.0,264.0,51.0,...,11741.257297,2430500.0,159220.039938,0.0,0.0,0.0,33.0,73.0,NaN,NaN


In [36]:
# import requests
# import pandas as pd

# # Etherscan API密钥
# api_key = '1RY32CCK7M9Z9AQPFZIMXB6HZE8PW2XMDN'

# # 获取最新区块号的函数
# def get_latest_block_number():
#     url = f"https://api.etherscan.io/api?module=proxy&action=eth_blockNumber&apikey={api_key}"
#     response = requests.get(url)
#     data = response.json()
#     if 'result' in data:
#         return int(data['result'], 16)
#     else:
#         print(f"Error fetching latest block number: {data['message']}")
#         return None

# # 获取指定区块交易的函数
# def get_block_transactions(block_number):
#     url = f"https://api.etherscan.io/api?module=proxy&action=eth_getBlockByNumber&tag={hex(block_number)}&boolean=true&apikey={api_key}"
#     response = requests.get(url)
#     data = response.json()
#     if 'result' in data:
#         return data['result']['transactions']
#     else:
#         print(f"Error fetching block transactions: {data['message']}")
#         return []

# # 获取多个区块交易的函数
# def fetch_all_transactions(start_block, end_block):
#     all_transactions = []
#     for block_number in range(start_block, end_block + 1):
#         transactions = get_block_transactions(block_number)
#         all_transactions.extend(transactions)
#     return pd.DataFrame(all_transactions)

# # 示例：获取最近10个区块的所有交易数据
# latest_block = get_latest_block_number()
# if latest_block:
#     start_block = latest_block - 10  # 获取最近10个区块
#     end_block = latest_block
#     transactions = fetch_all_transactions(start_block, end_block)

#     # 保存交易数据到CSV文件
#     csv_file = 'all_ethereum_transactions.csv'
#     if not transactions.empty:
#         transactions.to_csv(csv_file, index=False)
#         print(f"Transaction data has been saved to '{csv_file}'.")
#     else:
#         print("No transaction data retrieved.")
